In [13]:
import pandas as pd
import keras
from PIL import Image
from numpy import asarray
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from tqdm import trange

#Get rid of SettingWithCopyWarnings
pd.options.mode.chained_assignment = None  

In [2]:
#This .csv file comes from running some SQL queries on the iNaturalist open dataset.

roses = pd.read_csv('C:/Users/Sean Haight/Desktop/Coding/Erdos Project/Data Science/Flower Identification Project/Data/rawData/RosePictures.csv', sep = "\t")

In [16]:
'''
Here is code to create a batch file to download all the necessary images. 

n = 1000
m = 200
with open("C:/Users/Sean Haight/Desktop/Coding/Erdos Project/Data Science/Flower Identification Project/Notebooks/downloadRosePhotos.bat", "w") as file:

    for j in range(n):
        file.write('curl')
        for i in range(m):
            ind = i + m*j 
            if ind < len(roses['photo_url_square']):
                url = roses['photo_url_square'][ind]
                fileName = 'D:/Roses/' + str(roses['photo_id'][ind]) +'.' + roses['extension'][ind]
                file.write(' -o ' + fileName + ' ' + url)
        
        file.write('\n')
'''

In [3]:
#We need to label each rose images using our trained model

model = keras.models.load_model('../Models/roseModel.keras')

In [69]:
#First we add a file directory to each photo
path = "D:\Roses\\"
roses['fileName'] = path + roses['photo_id'].astype(str) + '.' + roses['extension']
roses['inBloom'] = None
n = 194
m = 1000
preds = []
#Loop through the images and add the model predictions to our dataframe
for j in trange(n):
    listOfImages = []
    exceptions = []
    for i in range(m):
        ind = i + m*j 
        if ind < len(roses):
            #Some of the images did not download properly or were missing, about 15/193,000
            try:
                imgDir = roses['fileName'][ind]
                img = load_img(imgDir)
                imgArray = img_to_array(img)

                #Rescale according to the model architecture
                imgArray = imgArray/255
                listOfImages.append(imgArray)
            except:
                exceptions.append(i)
        else: 
            break
    #If the model prediction is larger than .5 we assume that is an image of a flower.
    pred = (model.predict(np.array(listOfImages), verbose = 0) > .5).flatten()
    #Label the missing images with 'None'
    for e in exceptions:
        pred = np.insert(pred, e, None)
    roses['inBloom'][j*m:min((j + 1)*m, len(roses))] = pred

  1%|          | 1/194 [00:04<14:33,  4.52s/it]

D:\Roses\5453247.jpg


  1%|          | 2/194 [00:09<14:40,  4.58s/it]

999
319
1000


 22%|██▏       | 43/194 [03:03<10:31,  4.18s/it]

...

 95%|█████████▍| 184/194 [12:58<00:43,  4.31s/it]

D:\Roses\75314791.jpg


 95%|█████████▌| 185/194 [13:03<00:40,  4.56s/it]

999
381
1000


100%|██████████| 194/194 [13:44<00:00,  4.25s/it]


In [84]:
#Write our updated dataframe to file as a CSV
roses.to_csv('../Data/processedData/labeledBigRose.csv', sep='\t')